In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = '/home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data'
log_dir = '/home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data/logs'

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/hhl/code/machine_learning_tutorial/tensorflow_tutorial/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_true = tf.placeholder(tf.float32, [None, 10], name='y-input')

In [5]:
with tf.name_scope('input_shape'):
    image_shaped_input = tf.reshape(x,[-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input)

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [9]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act = tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('WX_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations

In [10]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability',keep_prob)
    droped = tf.nn.dropout(hidden1, keep_prob)
y_hat = nn_layer(droped, 500, 10, 'layer2', act=tf.identity)

In [11]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y_true)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'cross_entropy_1:0' shape=() dtype=string>

In [12]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediciton'):
        correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [13]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()

In [14]:
def feed_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x : xs, y_true : ys, keep_prob: k }

In [15]:
saver = tf.train.Saver()
for i in range(max_steps):
    if i % 10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary,i)
        print('Accuracy, at step %s: %s' % (i, acc))
    else:
        if i % 100 == 0:
            run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary,_ = sess.run([merged, train_step], feed_dict=feed_dict(True), options=run_options, run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step %.3d' % i)
            train_writer.add_summary(summary, i)
            saver.save(sess, log_dir + '/model.ckpt', i)
            print('Adding run metadata for ', i)
        else:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)
train_writer.close()
train_writer.close()

Accuracy, at step 0: 0.1147
Accuracy, at step 10: 0.7338
Accuracy, at step 20: 0.8359
Accuracy, at step 30: 0.848
Accuracy, at step 40: 0.878
Accuracy, at step 50: 0.8956
Accuracy, at step 60: 0.8979
Accuracy, at step 70: 0.905
Accuracy, at step 80: 0.9095
Accuracy, at step 90: 0.9107
Accuracy, at step 100: 0.9164
Accuracy, at step 110: 0.9126
Accuracy, at step 120: 0.9208
Accuracy, at step 130: 0.9206
Accuracy, at step 140: 0.9243
Accuracy, at step 150: 0.9266
Accuracy, at step 160: 0.9297
Accuracy, at step 170: 0.933
Accuracy, at step 180: 0.9303
Accuracy, at step 190: 0.9309
Accuracy, at step 200: 0.9363
Accuracy, at step 210: 0.9365
Accuracy, at step 220: 0.9381
Accuracy, at step 230: 0.9392
Accuracy, at step 240: 0.9428
Accuracy, at step 250: 0.9397
Accuracy, at step 260: 0.9316
Accuracy, at step 270: 0.9424
Accuracy, at step 280: 0.944
Accuracy, at step 290: 0.9458
Accuracy, at step 300: 0.9444
Accuracy, at step 310: 0.9454
Accuracy, at step 320: 0.9458
Accuracy, at step 330: 0.9